# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: 

My initial understanding of the supervised learning problem at hand is that it is one of classification. That is, given the data available and features provided, the learning algorithm will attempt to predict a binary response of whether the student is at risk of failing to graduate and needs an early intervention or not.

Because the prediction's output will be binary, that is, true or false, this is a problem of classification and not regression. A regression problem would be one in which the output is continuous.

**

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [19]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [20]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.keys()) - 1

# TODO: Calculate passing students
n_passed = sum(student_data["passed"]=="yes")

# TODO: Calculate failing students
n_failed = sum(student_data["passed"]=="no")

# TODO: Calculate graduation rate
grad_rate = 100.*n_passed/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [21]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [22]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [24]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
#X_train = None
#X_test = None
#y_train = None
#y_test = None

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, random_state=42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: 

The three supervised learning models that are going to be used for this particular problem are the following: Gaussian Naive Bayes, Decision Trees, and K-Nearest Neighbors.

Gaussian Naive Bayes:

Real World Applications: This particular algorithm has seen applications in spam filtering, such as Paul Graham's attempts (http://www.paulgraham.com/spam.html and http://www.paulgraham.com/better.html). Which leveraged the use of content based filters, that is, identifying probabilities of whether an email is spam based on specific features such as word occurrence in the body and header, size, links, etc. The implementation of the algorithm at hand on these situations is fairly straight forward and simple. One other application pertains more directly to the problem at hand, Kotsiantis, Pierrakeas, Pintelas (2003) used different Machine Learning Techniques in identifying the possibility of increased prevention of Student Dropouts in Distance Learning given data on the 'Informatics Course' of the Hellenic Open University. They found that the Naive Bayes algorithm was successfully used and yielded good results.

Strengths: The model's strengths are that it is of fairly easy and non-complex implementation. The feature probabilities allow for an understanding of the different influences each feature poses to the classification output, granting a somewhat less "black box" approach to the algorithm's inner working and a clearer picture of what features are more prone to influence the prediction. This would allow, for this specific case, a clearer picture as to what are the features that are influencing the most the student's passing/failing chance on the final exam, and hence make way for the possibility of identifying these features and improving them. Although not akin to the dataset at hand, the model tends to perform well with very large number of feature sets, and is very efficient at handling these larger datasets, which in turn is one of the reasons for its usage in cases where there is a need for an accurate but fast prediction with many features, like the case of text classification. 

Weaknesses: The model assumes conditionally independent features, meaning a specific feature input should not be probabilistically determined by some other feature value. In real world situations, this is not necessarily the case, as features tend to have a more complex nature and tend do not always be independent. An example stems from text classification as the combination of words might not necessarily share the same meaning or influence as the words do when separated. When the features are not independent, the algorithm's accuracy will tend to decrease.

Good Candidate: As demonstrated above, given the data at hand, the Naive Bayes model serves as a good candidate for this problem. The dataset is fairly small, meaning the algorithm should ideally be of less complexity, as to avoid overfitting. Given the objective of the study is to increase the ratio of students passing the final exam, it is important that the solution provides not only an accurate prediction of passing chances but also a clear indication as to what are the factors that influence this prediction. This way, the school board/directors would be able to identify what are the features that most influence this prediction and target them specifically. This way, not only the school is able to identify students that are likely to end up failing the final exam but make changes that reduce this statistic.

Decision Tree:

Real World Applications: One famous application of the decision tree is in BP's GasOIL system for separating gas and oil on offshore platforms (MIT Course Notes, Decision Trees, Cynthia Rudin, https://ocw.mit.edu/courses/sloan-school-of-management/15-097-prediction-machine-learning-and-statistics-spring-2012/lecture-notes/MIT15_097S12_lec08.pdf). The system in place had been "man-made" over a 10 year span and the new decision tree solution ended up outperforming field experts and saving BP millions of dollars. (1986). Because this model follows a very clear conception of decision making, it can be said that it is widely used in mundane situations, albeit unknowingly, for example in a doctor's thought process of investigating a disease, that is, asking a series of questions and eliminating possibilities based on the feedback or test, and arriving on some diagnosis.

Strengths: Because Decision Trees follow this fairly linear elimination process, which is replicated constantly on general decision making in a day to day basis, the interpretation for a non machine-learning person is intuitive and usually easy to follow, although more complex decision trees with many nodes can end up being fairly convoluted. In general though, this ease of interpretation allows for the identification of important features and more specifically, through the Information Gain formula, it is possible to create a decision tree that points toward features that are most likely to influence the outcome or prediction. This key feature identification is important in situations in which gaining an easy to understand insight on the data features is necessary and when the objective is not only to predict the outcome of some new data but to make changes to the system in place in order to increase the likelihood of some outcome, which is true for the case at hand.

Weaknesses: Decision Trees are prone to overfitting the data. In cases in which the dataset features are fairly complex, the algorithm will tend to overfit the training data and thus reduce the accuracy in which it predicts new data points, hence the trade-off on accuracy with "pruning" techniques is usually necessary. Given very large datasets and increased feature complexity, training a Decision Tree algorithm can also be very slow. Small modifications to the input data can issue very large changes to the tree, meaning the robustness and reliability of the algorithm can be compromised if the information the decision tree feeds from is not entirely precise. (http://www.brighthubpm.com/project-planning/106005-disadvantages-to-using-decision-trees/)

Good Candidate: Once more the decision for using this model was aimed at providing a more intuitive approach and a response to the school on important features that influence the final exam passing/failing outcome. Once again, the objective of this project would not only be to provide an accurate prediction for the passing chance of some new student given the feature values, but also a better understanding of the underlying effects that the different features have on the chances the student will pass the final exam or not. This way, this algorithm would be able to point the direction as to what are the features of interest that deserve being further looked at and targeted in order to increase the ratio of students passing the final exam. Given the data does not seem to be complex and the amount of data is fairly small, Decision Trees would not pose a problem of large memory consumption and convoluted branching out interpretations, although the issue of overfitting would be important to analyze.

K-Nearest Neighbors:

Real World Application: k-NN has been used in several different fields, usually linked to some kind of pattern based classification problem, like for example on the breast cancer diagnosis problem, Sarkar, Leong (2000), which demonstrated that k-NN provided classification results that were 1.17% better than the best results known for the problem at the time. One other application was in the prediction of economic events, as demonstrated on Bafandeh, Bolandraftar (2013) which followed this pattern based intuition to determine the effects of companies' financial distress on stakeholders, also demonstrating positive results. Other pattern based approaches like recommendation systems (Music, Movies, Series, Books) can also be tackled with a k-NN Classifier, as the objective is to identify similar data points and draw practical inferences on the predicted user tastes.

Strengths: Following suit to the previously chosen algorithms, the k-NN classifier is also a fairly intuitive, non "black-box" approach, that allows for a simpler implementation and understanding of the reasons that lead to some prediction of an outcome based on the features. In this case, the algorithm predicts an outcome based on the most similar k points. Because the computational part of k-NN is held in the classification of a data point, the performance of the algorithm is fairly good, although for large training data sets, the computation can be very taxing. However, this is not true for the case at hand.

Weaknesses: Determining the distance function that is most appropriate to the problem can be quite unclear, and given this choice might greatly influence the predictive power of the algorithm, this can produce a weak accuracy. One other problem is that all features contribute to the similarity between the data points, and if the weights are not properly tuned, the model might incorrectly predict classifications. As mentioned previously, for large datasets, the computational cost can be very high.

Good Candidate: Following the importance attributed to the ease of interpretation of results and explanatory power of the algorithm regarding important features, k-NN seems like a good candidate for this specific study. The dataset available is not large and complex, which implies that one of the model's weaknesses, computational cost for large datasets, should not be a problem. Given the importance of prediction and understanding of feature importance, this model should be rather intuitive. Seeing as the problem at hand fits within a pattern classification problem, that is, identifying students more prone to failing the final exam should be a result of some underlying pattern, this model should be able to correctly identify these trends or patterns in the data and make accurate predictions based on the given features.

Summary: From the problem's description I set out to identify models that would be good at two specific tasks, working with small datasets and still presenting fairly accurate results, and demonstrating the underlying cause for the prediction in an intuitive way, that is, not being a "black box" kind of prediction. Therefore, although there are other candidate models that could surely pose good results, I opted for these three classifiers.

**

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [25]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [41]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = DecisionTreeClassifier()
clf_C = KNeighborsClassifier()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for classifier in [clf_A, clf_B, clf_C]:
    for X, y in zip([X_train_100, X_train_200, X_train_300], [y_train_100, y_train_200, y_train_300]):
        train_predict(classifier, X, y, X_test, y_test)
        

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0119 seconds
Made predictions in 0.0043 seconds.
F1 score for training set: 0.8467.
Made predictions in 0.0009 seconds.
F1 score for test set: 0.8029.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0035 seconds
Made predictions in 0.0016 seconds.
F1 score for training set: 0.8406.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.7244.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0024 seconds
Made predictions in 0.0014 seconds.
F1 score for training set: 0.8038.
Made predictions in 0.0012 seconds.
F1 score for test set: 0.7634.
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0045 seconds
Made predictions in 0.0013 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.6102.
Training a DecisionTreeClassifier using a training set size of 200. 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0119                  |0.0009                  |0.8467            |0.8029|
| 200               |        0.0035          |      0.0008            |    0.8406        |    0.7244       |
| 300               |          0.0024       |      0.0012          |     0.8038       |    0.7634      |

** Classifer 2 - Decision Tree**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0045            |       0.0006          |     1.0000        |     0.6102      |
| 200               |     0.0051            |       0.0006          |    1.0000       |     0.7407     |
| 300               |        0.0051         |      0.0009        |     1.0000      |      0.6929     |

** Classifer 3 - k-NN**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0014       |         0.0020     |      0.8060    |     0.7246      |
| 200               |         0.0012        |     0.0026          |  0.8800          |      0.7692     |
| 300               |          0.0020        |        0.0030        |   0.8809       |     0.7801     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: 

In order to compare the models and come up with the most accurate and robust one, the mean F1 Score on the test was considered, with the following results presented:

Gaussian: 0.76357
Decision Tree: 0.68127
k-NN: 0.75797

With the Gaussian Naive Bayes algorithm coming out on top, and therefore considered the best model for this particular problem. This algorithm also had the maximum F1 value obtained, at 0.8029.

Regarding performance and the limited resources available, given the dataset is limited, the training and prediction time for all models were fairly small, with k-NN expectedly, coming out a bit slower when predicting the test values. Hence the difference would be minimal considering the amount of data available. However k-NN seems to be also the model with the greatest increase in time consumption with increases to the number of data points tested, hence as the database increases, it could be a more costly alternative. Although the Decision Tree shows good performance, the accuracy attributed by the F1 score is lower compared to the other models. the Gaussian Naive Bayes model comes out as an intermediate model in terms of performance but with the highest average F1 scores.

**

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: 

The way the Naive Bayes algorithm works is that we get all the data points provided and separate them in two main subsets, training, around 75%, and testing, the remaining 25%. With the training set, we then run the algorithm on this training set and from the different combination of features present in the data, the model identifies what are the weights associated with each feature. That is, how much does each feature influence the outcome of whether a student passes the final exam or not. We then use this combination of probabilities that was captured by the algorithm from the training set and test it on the remaining testing subset. This way we can validate the algorithm and see how well it performs on this new set of data points, since we already know what the results should be of whether the student passed or not.

By performing these steps the algorithm attempts to identify a trend in the data. The Naive Bayes algorithm's goal is to therefore, fine tune these weights for each feature. This opens way for the possibility of getting strong insights from what the data shows, and give an idea of what are the factors that most contribute towards a student passing the final exam or not and giving a good prediction, given a student's present feature values, of whether this student would pass or not, and thus being able to act upon it.

**

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [42]:
# NOTE: Beacause the final chosen model, GaussianNB does not take any parameters, I've chosen to see if I can improve
# the f1 score for the lowest scoring Model, Decision Tree, in order to see the power of GridSearchCV in identifying 
# the best performing parameters.


# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# TODO: Create the parameters list you wish to tune
parameters = {'min_samples_split':[1, 2, 3, 4, 5, 6],
              'min_samples_leaf': [1, 2, 3, 4, 5, 6],
              'max_depth': [1, 2, 3, 4, 5, 6]}

# TODO: Initialize the classifier
clf = DecisionTreeClassifier(random_state=42)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score,  pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0005 seconds.
Tuned model has a training F1 score of 0.8299.
Made predictions in 0.0006 seconds.
Tuned model has a testing F1 score of 0.8000.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: 

*As mentioned in the coding portion, the chosen model for this part was Decision Trees, as Gaussian Naive Bayes does not take parameters.

The result of GridSearch Cross Validation on the Decision Tree model showed a final training F1 score of 0.8299, down from 1.000, which indicates the model seems to not be overfitting the training data as much, and a final testing F1 score of 0.8000, up from an average of 0.68127, which means the tuned model performs much better than the untuned model.

The following parameters were the ones used: {'min_samples_split': 1, 'max_depth': 2, 'min_samples_leaf': 6}

**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.